# HSE Summer School NLP&DA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Utility functions

In [26]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

def transformAnnotation(number):
    """
    Given number loads txt file with annotation 
    Returns DataFrame with transformed annotation
    """
    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    return d


def transformAnswer(number):
    """
    Given number loads txt file with answer 
    Returns DataFrame with transformed answer
    """
    answ = loadAnswer(number)
    answ = answ.split("\n")

    for i in range(len(answ)):
        answ[i] = answ[i].split(",")

    answ = pd.DataFrame(answ)
    answ.columns = ["entity_1", "entity_2", "attitude", "time"]
    answ.dropna(inplace=True)
    answ.time = answ.time.apply(lambda x: x.strip("\r"))
    
    return answ

![](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)

In [245]:
d = transformAnnotation(2)
a = transformAnswer(2)

In [246]:
d[d.entity!="Unknown"].head()

,entity,entity_car,pos_1,pos_2
0,Author,PER,1,7
2,Author,PER,22,28
4,Россия,LOC,39,45
5,Сирии,LOC,72,77
6,Author,PER,81,87


In [249]:
a.head()

,entity_1,entity_2,attitude,time
0,Сирия,Россия,pos,current
1,США,Асад,neg,past
2,Москва,Израиль,pos,current
3,Израиль,Россия,pos,current
4,Сирия,Израиль,neg,past


In [38]:
print(loadText(2))

{Author, Unknown} 

{Author, Unknown} Россия и кратковременный успех в Сирии

{Author, Unknown} inosmi.ruПосмотреть оригиналапрель 5-го, 2016

{Author, Unknown} Россия реализовала свои цели в Сирии, по крайней мере в краткосрочной перспективе. 
{Author, Unknown} Что будет дальше, пока неясно. 
{Author, Unknown} Точно можно сказать одно: подавляющее большинство сирийских граждан не смогут забыть российского участия в войне, которая была направлена против всего сирийского народа.


{Author, Unknown} Что касается Москвы, то будущее Сирии пока слабо представляется без участия России, особенно если принять во внимание, что сирийский конфликт имеет три уровня: иранский, американский и израильский.


{Author, Unknown} Иран окончательно смирился с тем, что судьба Башара Асада полностью зависит от решения Кремля. 
{Author, Unknown} Россия была призвана спасти то, что осталось от алавитского сирийского режима, сохранение которого представляет для Ирана особую важность.


{Author, Unknown} Если б